In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm as tqdm
import shutil

parameters

In [2]:
cleaned_fine_csv_fileName = "cleaned_metadata.csv"
data_root = "/raid/elhamod/Fish"#"/data/BGNN_data/INHS_cropped"
source_suffix_list=["cropped2/FMNH", "cropped2/OSUM", "cropped2/INHS", "cropped2/JFBM", "cropped2/UWZM"]
suffix = "Curated3/Hard/curated_30_50"
training_count=0.64
validation_count=0.16

constants

In [3]:
# metadata table headers.
fine_csv_fileName_header = "fileName"
fine_csv_scientificName_header = "scientificName"
fine_csv_Coarse_header = "Genus"
fine_csv_Family_header = "Family"
fine_csv_usedColumns = [fine_csv_fileName_header,
                          fine_csv_scientificName_header,
                          fine_csv_Coarse_header,
                          fine_csv_Family_header]

Read CSV

In [4]:
cleaned_fine_csv_fileName_full_path = os.path.join(data_root, suffix, cleaned_fine_csv_fileName)
fine_csv = pd.read_csv(cleaned_fine_csv_fileName_full_path, delimiter='\t', index_col=fine_csv_fileName_header, usecols=fine_csv_usedColumns)

Split data into train/val/test

In [5]:
fine_labels = fine_csv[fine_csv_scientificName_header].unique()

In [6]:
fine_csv_size = len(fine_csv.index)
labels = fine_csv[fine_csv_scientificName_header].to_numpy()

train_indices, test_indices = train_test_split(range(fine_csv_size), test_size= 1-training_count-validation_count, 
                                                            stratify=labels)
labels_sub = labels[train_indices]
train_indices, val_indices = train_test_split(train_indices, test_size= validation_count/(validation_count+training_count), 
                                                            stratify=labels_sub)

Create directories for each subset

In [7]:
def create_dataset(experiment_name, indices, subset_name, csv):
    # create directiory
    folder_name = os.path.join(data_root, suffix, subset_name)
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    
    for i in tqdm(indices):
        row = csv.iloc[i]
        fine_label = row[fine_csv_scientificName_header]
        file_name = csv.index[i]

        # create fine_label directory if not found
        sub_folder_name = os.path.join(data_root, suffix, subset_name, fine_label)
        if not os.path.exists(sub_folder_name):
            os.makedirs(sub_folder_name)
            
        # copy file
        for source in source_suffix_list:
            try:
                fileFullName = os.path.join(sub_folder_name,file_name)
                newFile = shutil.copy(os.path.join(data_root, source, 'images', file_name), fileFullName)
                break
            except:
                pass
        

In [8]:
create_dataset(suffix, train_indices, 'train', fine_csv)

In [9]:
create_dataset(suffix, val_indices, 'val', fine_csv)

In [10]:
create_dataset(suffix, test_indices, 'test', fine_csv)